In [ ]:
import tensorflow as tf

import sys
import os
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
np.random.seed(42)
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [ ]:
import pandas as pd
import glob

Cases_sequence_all_files = glob.glob("/Cases_sequence_*.csv")

li = []

for filename in Cases_sequence_all_files:
    if "fold1" in filename:
        continue
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

Cases_sequence_frame = pd.concat(li, axis=0, ignore_index=True,sort=True)

Cases_features_all_files = glob.glob("/Cases_features_*.csv")

li = []

for filename in Cases_features_all_files:
    if "fold1" in filename:
        continue

    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

Cases_features_frame = pd.concat(li, axis=0, ignore_index=True,sort=True)

Controls_sequence_all_files = glob.glob("/Controls_sequence_*.csv")

li = []

for filename in Controls_sequence_all_files:
    if "fold1" in filename:
        continue

    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

Controls_sequence_frame = pd.concat(li, axis=0, ignore_index=True,sort=True)

Controls_features_all_files = glob.glob("/Controls_features_*.csv")

li = []

for filename in Controls_features_all_files:
    if "fold1" in filename:
        continue

    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

Controls_features_frame = pd.concat(li, axis=0, ignore_index=True,sort=True)


data_for_rnn_train_feat_PAD = Cases_features_frame
data_for_rnn_train_feat_CONT = Controls_features_frame
data_for_rnn_train_seq_PAD = Cases_sequence_frame
data_for_rnn_train_seq_CONT = Controls_sequence_frame



In [ ]:
fold_num = 'fold1'
fold_num_print = 'Fold num 1'
data_for_rnn_test_seq_PAD=pd.read_csv('/Cases_sequence_'+fold_num+'.csv',header=0, skip_blank_lines=True,low_memory=False)
data_for_rnn_test_feat_PAD=pd.read_csv('/Cases_features_'+fold_num+'.csv',header=0, skip_blank_lines=True,low_memory=False)
data_for_rnn_test_seq_CONT=pd.read_csv('/Controls_sequence_'+fold_num+'.csv',header=0, skip_blank_lines=True,low_memory=False)
data_for_rnn_test_feat_CONT=pd.read_csv('/Controls_features_'+fold_num+'.csv',header=0, skip_blank_lines=True,low_memory=False)



In [ ]:
import numpy as np

np.unique(data_for_rnn_test_feat_CONT['observation_unique'])
le = LabelEncoder()
le.fit(data_for_rnn_test_feat_CONT['observation_unique'])
le.transform(data_for_rnn_test_feat_CONT['observation_unique'])


In [ ]:
data_for_rnn_train_seq = pd.concat([data_for_rnn_train_seq_PAD,data_for_rnn_train_seq_CONT],axis=0)
data_for_rnn_train_feat = pd.concat([data_for_rnn_train_feat_PAD,data_for_rnn_train_feat_CONT],axis=0)

In [ ]:
data_for_rnn_test_seq = pd.concat([data_for_rnn_test_seq_PAD,data_for_rnn_test_seq_CONT],axis=0)
data_for_rnn_test_feat = pd.concat([data_for_rnn_test_feat_PAD,data_for_rnn_test_feat_CONT],axis=0)

In [ ]:
test_data_sample = pd.DataFrame()
test_data_sample['patient'] = data_for_rnn_test_seq['person_id']
test_data_sample['code'] = data_for_rnn_test_seq['concept_id']
test_data_sample['type'] = data_for_rnn_test_seq['type']
test_data_sample['rec'] = data_for_rnn_test_seq['recency']
test_data_sample['year'] = data_for_rnn_test_seq['year']
test_data_sample['days'] = data_for_rnn_test_seq['distance_days']
test_data_sample['ag'] = data_for_rnn_test_seq['age']
test_data_sample

In [ ]:
train_data_sample = pd.DataFrame()
train_data_sample['patient'] = data_for_rnn_train_seq['person_id']
train_data_sample['code'] = data_for_rnn_train_seq['concept_id']
train_data_sample['type'] = data_for_rnn_train_seq['type']
train_data_sample['rec'] = data_for_rnn_train_seq['recency']
train_data_sample['year'] = data_for_rnn_train_seq['year']
train_data_sample['days'] = data_for_rnn_train_seq['distance_days']
train_data_sample['ag'] = data_for_rnn_train_seq['age']
train_data_sample

In [ ]:
test_data_sample_user = pd.DataFrame()
test_data_sample_user['patient'] = data_for_rnn_test_feat['person_id']
test_data_sample_user['f1'] = data_for_rnn_test_feat['age']
test_data_sample_user['f2'] = data_for_rnn_test_feat['race']
test_data_sample_user['f3'] = data_for_rnn_test_feat['gender']
test_data_sample_user['f4'] = data_for_rnn_test_feat['condition_unique']
test_data_sample_user['f5'] = data_for_rnn_test_feat['procedure_unique']
test_data_sample_user['f6'] = data_for_rnn_test_feat['observation_unique']
test_data_sample_user['label'] = data_for_rnn_test_feat['label']

test_data_sample_user

In [ ]:
train_data_sample_user = pd.DataFrame()
train_data_sample_user['patient'] = data_for_rnn_train_feat['person_id']
train_data_sample_user['f1'] = data_for_rnn_train_feat['age']
train_data_sample_user['f2'] = data_for_rnn_train_feat['race']
train_data_sample_user['f3'] = data_for_rnn_train_feat['gender']
train_data_sample_user['f4'] = data_for_rnn_train_feat['condition_unique']
train_data_sample_user['f5'] = data_for_rnn_train_feat['procedure_unique']
train_data_sample_user['f6'] = data_for_rnn_train_feat['observation_unique']
train_data_sample_user['label'] = data_for_rnn_train_feat['label']

train_data_sample_user

In [ ]:
pd_test_train = pd.concat([train_data_sample['code'],test_data_sample['code']])
pd_test_train

In [ ]:
le = LabelEncoder()
le.fit(pd.concat([train_data_sample['code'],test_data_sample['code']]))
le_tp = LabelEncoder()
le_tp.fit(pd.concat([train_data_sample['type'],test_data_sample['type']]))
le_rec = LabelEncoder()
le_rec.fit(pd.concat([train_data_sample['rec'],test_data_sample['rec']]))
le_yr = LabelEncoder()
le_yr.fit(pd.concat([train_data_sample['year'],test_data_sample['year']]))
le_dy = LabelEncoder()
le_dy.fit(pd.concat([train_data_sample['days'],test_data_sample['days']]))
le_ag = LabelEncoder()
le_ag.fit(pd.concat([train_data_sample['ag'],test_data_sample['ag']]))
le_f1 = LabelEncoder()
le_f1.fit(pd.concat([train_data_sample_user['f1'],test_data_sample_user['f1']]))
le_f2 = LabelEncoder()
le_f2.fit(pd.concat([train_data_sample_user['f2'],test_data_sample_user['f2']]))
le_f3 = LabelEncoder()
le_f3.fit(pd.concat([train_data_sample_user['f3'],test_data_sample_user['f3']]))
le_f4 = LabelEncoder()
le_f4.fit(pd.concat([train_data_sample_user['f4'],test_data_sample_user['f4']]))
le_f5 = LabelEncoder()
le_f5.fit(pd.concat([train_data_sample_user['f5'],test_data_sample_user['f5']]))
le_f6 = LabelEncoder()
le_f6.fit(pd.concat([train_data_sample_user['f6'],test_data_sample_user['f6']]))

train_data_sample['code'] = le.transform( train_data_sample['code'])
train_data_sample['type'] = le_tp.transform( train_data_sample['type'])
train_data_sample['rec'] = le_rec.transform( train_data_sample['rec'])
train_data_sample['year'] = le_yr.transform( train_data_sample['year'])
train_data_sample['days'] = le_dy.transform( train_data_sample['days'])
train_data_sample['ag'] = le_ag.transform( train_data_sample['ag'])

train_data_sample_user['f1'] = le_f1.transform( train_data_sample_user['f1'])
train_data_sample_user['f2'] = le_f2.transform( train_data_sample_user['f2'])
train_data_sample_user['f3'] = le_f3.transform( train_data_sample_user['f3'])
train_data_sample_user['f4'] = le_f4.transform( train_data_sample_user['f4'])
train_data_sample_user['f5'] = le_f5.transform( train_data_sample_user['f5'])
train_data_sample_user['f6'] = le_f6.transform( train_data_sample_user['f6'])


test_data_sample['code'] = le.transform( test_data_sample['code'])
test_data_sample['type'] = le_tp.transform( test_data_sample['type'])
test_data_sample['rec'] = le_rec.transform( test_data_sample['rec'])
test_data_sample['year'] = le_yr.transform( test_data_sample['year'])
test_data_sample['days'] = le_dy.transform( test_data_sample['days'])
test_data_sample['ag'] = le_ag.transform( test_data_sample['ag'])

test_data_sample_user['f1'] = le_f1.transform( test_data_sample_user['f1'])
test_data_sample_user['f2'] = le_f2.transform( test_data_sample_user['f2'])
test_data_sample_user['f3'] = le_f3.transform( test_data_sample_user['f3'])
test_data_sample_user['f4'] = le_f4.transform( test_data_sample_user['f4'])
test_data_sample_user['f5'] = le_f5.transform( test_data_sample_user['f5'])
test_data_sample_user['f6'] = le_f6.transform( test_data_sample_user['f6'])


train_data_sample,test_data_sample,train_data_sample_user,test_data_sample_user

In [ ]:
test_data_sample_agg = pd.DataFrame()
test_data_sample_agg['patient']=test_data_sample.groupby(['patient'],as_index=False).agg({'code':lambda x: list(x)})['patient']
test_data_sample_agg['code']=test_data_sample.groupby(['patient'],as_index=False).agg({'code':lambda x: list(x)})['code']
test_data_sample_agg['type']=test_data_sample.groupby(['patient'],as_index=False).agg({'type':lambda x: list(x)})['type']
test_data_sample_agg['rec']=test_data_sample.groupby(['patient'],as_index=False).agg({'rec':lambda x: list(x)})['rec']
test_data_sample_agg['year']=test_data_sample.groupby(['patient'],as_index=False).agg({'year':lambda x: list(x)})['year']
test_data_sample_agg['days']=test_data_sample.groupby(['patient'],as_index=False).agg({'days':lambda x: list(x)})['days']
test_data_sample_agg['ag']=test_data_sample.groupby(['patient'],as_index=False).agg({'ag':lambda x: list(x)})['ag']
test_data_sample_agg

In [ ]:
train_data_sample_agg = pd.DataFrame()
train_data_sample_agg['patient']=train_data_sample.groupby(['patient'],as_index=False).agg({'code':lambda x: list(x)})['patient']
train_data_sample_agg['code']=train_data_sample.groupby(['patient'],as_index=False).agg({'code':lambda x: list(x)})['code']
train_data_sample_agg['type']=train_data_sample.groupby(['patient'],as_index=False).agg({'type':lambda x: list(x)})['type']
train_data_sample_agg['rec']=train_data_sample.groupby(['patient'],as_index=False).agg({'rec':lambda x: list(x)})['rec']
train_data_sample_agg['year']=train_data_sample.groupby(['patient'],as_index=False).agg({'year':lambda x: list(x)})['year']
train_data_sample_agg['days']=train_data_sample.groupby(['patient'],as_index=False).agg({'days':lambda x: list(x)})['days']
train_data_sample_agg['ag']=train_data_sample.groupby(['patient'],as_index=False).agg({'ag':lambda x: list(x)})['ag']
train_data_sample_agg

In [ ]:
train_data=pd.DataFrame(train_data_sample_user.merge(train_data_sample_agg, how='inner', right_on='patient',left_on='patient'))
train_data,np.unique(train_data.label.values)

In [ ]:
test_data=pd.DataFrame(test_data_sample_user.merge(test_data_sample_agg, how='inner', right_on='patient',left_on='patient'))
test_data

In [ ]:
RF_person_ids_order=pd.read_csv('/testids_'+fold_num+'.csv',header=0, skip_blank_lines=True,low_memory=False)

test_data=pd.DataFrame(RF_person_ids_order.merge(test_data, how='inner', right_on='patient',left_on='test_person_id')).drop_duplicates('test_person_id')

test_data, RF_person_ids_order

In [ ]:
data_for_rnn_train = train_data
data_for_rnn_test = test_data
data_for_rnn_train,data_for_rnn_test

In [ ]:

#perform the padding for both prods seq and rec seq

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_input_length = 300
padding_value_c=len(le.classes_)+1
padding_value_t=len(le_tp.classes_)+1
padding_value_r=len(le_rec.classes_)+1
padding_value_y=len(le_yr.classes_)+1
padding_value_d=len(le_dy.classes_)+1
padding_value_a=len(le_ag.classes_)+1
input_sequences_train_code = pad_sequences(data_for_rnn_train['code'], maxlen=max_input_length, padding='pre',value=padding_value_c)
input_sequences_train_type = pad_sequences(data_for_rnn_train['type'], maxlen=max_input_length, padding='pre',value=padding_value_t)
input_sequences_train_rec = pad_sequences(data_for_rnn_train['rec'], maxlen=max_input_length, padding='pre',value=padding_value_r)
input_sequences_train_year = pad_sequences(data_for_rnn_train['year'], maxlen=max_input_length, padding='pre',value=padding_value_y)
input_sequences_train_days = pad_sequences(data_for_rnn_train['days'], maxlen=max_input_length, padding='pre',value=padding_value_d)
input_sequences_train_ag = pad_sequences(data_for_rnn_train['ag'], maxlen=max_input_length, padding='pre',value=padding_value_a)
input_sequences_test_code = pad_sequences(data_for_rnn_test['code'], maxlen=max_input_length, padding='pre',value=padding_value_c)
input_sequences_test_type = pad_sequences(data_for_rnn_test['type'], maxlen=max_input_length, padding='pre',value=padding_value_t)
input_sequences_test_rec = pad_sequences(data_for_rnn_test['rec'], maxlen=max_input_length, padding='pre',value=padding_value_r)
input_sequences_test_year = pad_sequences(data_for_rnn_test['year'], maxlen=max_input_length, padding='pre',value=padding_value_t)
input_sequences_test_days = pad_sequences(data_for_rnn_test['days'], maxlen=max_input_length, padding='pre',value=padding_value_d)
input_sequences_test_ag = pad_sequences(data_for_rnn_test['ag'], maxlen=max_input_length, padding='pre',value=padding_value_a)
output_sequences_train = data_for_rnn_train['label']
output_sequences_test = data_for_rnn_test['label']


In [ ]:
output_sequences_train = data_for_rnn_train['label']
output_sequences_test = data_for_rnn_test['label']


In [ ]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_train = np.array(output_sequences_train).reshape(len(output_sequences_train), 1)
integer_encoded_test = np.array(output_sequences_test).reshape(len(output_sequences_test), 1)


integer_encoded_train_test = np.concatenate((integer_encoded_train,integer_encoded_test),axis=0)


onehot_encoder.fit(integer_encoded_train_test)

onehot_OUTPUT_train =onehot_encoder.transform(integer_encoded_train)
onehot_OUTPUT_test =onehot_encoder.transform(integer_encoded_test)


In [ ]:

from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.grid_search import GridSearchCV
import sklearn
import pandas as pd



import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
np.random.seed(42)

import pandas
import random

from sklearn.preprocessing import LabelEncoder

import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

In [ ]:
output_vocab_size = len(onehot_encoder.categories_[0])
output_vocab_size


In [ ]:
input_vocab_size_code = len(le.classes_)
input_vocab_size_type = len(le_tp.classes_)
input_vocab_size_rec = len(le_rec.classes_)
input_vocab_size_year = len(le_yr.classes_)
input_vocab_size_days = len(le_dy.classes_)
input_vocab_size_prod_age = len(le_ag.classes_)


In [ ]:
input_vocab_size_f1 = len(le_f1.classes_)
input_vocab_size_f2 = len(le_f2.classes_)
input_vocab_size_f3 = len(le_f3.classes_)
input_vocab_size_f4 = len(le_f4.classes_)
input_vocab_size_f5 = len(le_f5.classes_)
input_vocab_size_f6 = len(le_f6.classes_)



In [ ]:
! pip install keras_self_attention

In [ ]:

from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.layers import LSTM # new! 
from tensorflow.keras.layers import  Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import SimpleRNN 
from tensorflow.keras.layers import GRU  
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import RNN # new! 
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LeakyReLU
import keras.backend as K
from tensorflow.keras.layers import BatchNormalization

from keras_self_attention import SeqSelfAttention

emb_size_c=100
emb_size_t=5
emb_size_r=10
emb_size_y=10
emb_size_d=10
emb_size_a=10
hidden_units_seq=600
hidden_units_feat=200

emb_size_f1=25
emb_size_f2=10
emb_size_f3=2
emb_size_f4=10
emb_size_f5=10
emb_size_f6=10

inp_c = Input(shape=(max_input_length,))
inp_t = Input(shape=(max_input_length,))
inp_r = Input(shape=(max_input_length,))
inp_y = Input(shape=(max_input_length,))
inp_d = Input(shape=(max_input_length,))
inp_a = Input(shape=(max_input_length,))

inp_f1 = Input(shape=(1,))
inp_f2 = Input(shape=(1,))
inp_f3 = Input(shape=(1,))
inp_f4 = Input(shape=(1,))
inp_f5 = Input(shape=(1,))
inp_f6 = Input(shape=(1,))

emb_c = Embedding(padding_value_c+1,emb_size_c, input_length=max_input_length)(inp_c)
y1 = Reshape((emb_size_c,max_input_length))(emb_c)

lstm_y1= LSTM(hidden_units_seq, dropout = 0.3, recurrent_dropout = 0.3)(y1)

emb_t = Embedding(padding_value_t+1,emb_size_t, input_length=max_input_length)(inp_t)
y2 = Reshape((emb_size_t,max_input_length))(emb_t)

emb_r = Embedding(padding_value_r+1,emb_size_r, input_length=max_input_length)(inp_r)
y3 = Reshape((emb_size_r,max_input_length))(emb_r)

emb_y = Embedding(padding_value_y+1,emb_size_y, input_length=max_input_length)(inp_y)
y4 = Reshape((emb_size_y,max_input_length))(emb_y)

emb_d = Embedding(padding_value_d+1,emb_size_d, input_length=max_input_length)(inp_d)
y5 = Reshape((emb_size_d,max_input_length))(emb_d)

emb_a = Embedding(padding_value_a+1,emb_size_a, input_length=max_input_length)(inp_a)
y6 = Reshape((emb_size_a,max_input_length))(emb_a)

x=concatenate([y1
               , 
               y2,
               y3
               ,
               y4,
               y5,
               y6
              ],axis=1)


lstm_l1= LSTM(hidden_units_seq, dropout = 0.5, recurrent_dropout = 0.5)(y1)


x_flat = Flatten()(x)

relu_x= Dense(50, activation='relu')(x_flat)

x_flat_flat = Flatten()(relu_x)

y1_flat = Flatten()(y1)

relu_y1= Dense(50, activation='relu')(y1_flat)

y1_flat_flat = Flatten()(relu_y1)


emb_f1 = Embedding(input_vocab_size_f1+1,emb_size_f1, input_length=1)(inp_f1)
f1 = Reshape((emb_size_f1,1))(emb_f1)

emb_f2 = Embedding(input_vocab_size_f2+1,emb_size_f2, input_length=1)(inp_f2)
f2 = Reshape((emb_size_f2,1))(emb_f2)

emb_f3 = Embedding(input_vocab_size_f3+1,emb_size_f3, input_length=1)(inp_f3)
f3 = Reshape((emb_size_f3,1))(emb_f3)

emb_f4 = Embedding(input_vocab_size_f4+1,emb_size_f4, input_length=1)(inp_f4)
f4 = Reshape((emb_size_f4,1))(emb_f4)

emb_f5 = Embedding(input_vocab_size_f5+1,emb_size_f5, input_length=1)(inp_f5)
f5 = Reshape((emb_size_f5,1))(emb_f5)

emb_f6 = Embedding(input_vocab_size_f6+1,emb_size_f6, input_length=1)(inp_f6)
f6 = Reshape((emb_size_f6,1))(emb_f6)



f1_2_conc = concatenate([f1
                         ,f2
                         ,f3
                         ,f4,f5,f6
                        ],axis=1)

flatten_f1_2_conc = Flatten()(f1_2_conc)
relu_f1_2 = Dense(hidden_units_feat, activation='relu')(flatten_f1_2_conc)

flatten_f1 = Flatten()(f1)
relu_f1 = Dense(hidden_units_feat, activation='relu')(flatten_f1)



lstm1_f = concatenate([
                        lstm_y1
                       , relu_f1_2
],axis=1)
predictions = Dense(2, activation='sigmoid')(lstm1_f)

model = Model(
    inputs=[inp_c
            ,
            inp_t,
            inp_r,
            inp_y,
            inp_d,
            inp_a
            ,
            inp_f1
            ,inp_f2
            ,inp_f3
            ,inp_f4,inp_f5,inp_f6
           ],
    outputs=predictions
)


sp = tf.keras.metrics.SpecificityAtSensitivity(0.55,200)
sp.update_state([0, 0, 0, 1, 1], [0, 0.3, 0.8, 0.3, 0.8])
sn = tf.keras.metrics.SensitivityAtSpecificity(0.5,200)
sn.update_state([0, 0, 0, 1, 1], [0, 0.3, 0.8, 0.3, 0.8])
tp=tf.keras.metrics.TruePositives()
tn = tf.keras.metrics.TrueNegatives()
fp = tf.keras.metrics.FalsePositives()
fn = tf.keras.metrics.FalseNegatives()


model.compile(loss='binary_crossentropy', optimizer='adam', 
              metrics=[
                  tf.keras.metrics.AUC(
    num_thresholds=200,
    curve="ROC",
    summation_method="interpolation",
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    label_weights=None,
),
                       sp, #see definition of the SpecificityAtSensitivity
                  sn #see definition of the SensitivityAtSpecificity
                  ,tn,tp,fn,fp
                      ])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=2, mode='min')


In [ ]:
import time

model.fit([pd.DataFrame(input_sequences_train_code)
           ,
           pd.DataFrame(input_sequences_train_type),
           pd.DataFrame(input_sequences_train_rec),
           pd.DataFrame(input_sequences_train_year),
           pd.DataFrame(input_sequences_train_days),
           pd.DataFrame(input_sequences_train_ag),
           pd.DataFrame(data_for_rnn_train['f1']),
           pd.DataFrame(data_for_rnn_train['f2']),
           pd.DataFrame(data_for_rnn_train['f3'])
           ,
           pd.DataFrame(data_for_rnn_train['f4']),
           pd.DataFrame(data_for_rnn_train['f5']),
           pd.DataFrame(data_for_rnn_train['f6'])
          ], onehot_OUTPUT_train,
          epochs=20, batch_size =64
          , 
          
          callbacks=[earlyStopping],
          validation_split=0.1,
          verbose=1, shuffle=True
         )


In [ ]:
import numpy as np
np.random.seed(10)

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.calibration import calibration_curve


y_pred_keras = model.predict([pd.DataFrame(input_sequences_test_code)
                              ,
           pd.DataFrame(input_sequences_test_type),
           pd.DataFrame(input_sequences_test_rec),
           pd.DataFrame(input_sequences_test_ag),
           pd.DataFrame(data_for_rnn_test['f1']),
           pd.DataFrame(data_for_rnn_test['f2']),
           pd.DataFrame(data_for_rnn_test['f3'])
                              ,
           pd.DataFrame(data_for_rnn_test['f4']),
           pd.DataFrame(data_for_rnn_test['f5']),
           pd.DataFrame(data_for_rnn_test['f6'])
                             ])
